## Syntax-dependent trace

Head attribution

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3

!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
torch.set_grad_enabled(False)

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/starcoder"

check_devs()
torch.cuda.is_available()

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84979089408 used for device 0, reserved 0


True

In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

Loading checkpoint shards: 100%|██████████| 7/7 [00:27<00:00,  3.98s/it]


62136934400 / 84979089408 used for device 0, reserved 62138613760


## Test Trace


In [4]:
prompts = ["""print("hello """]


txt, ret_dict = lm.trace_generate(
    prompts,
    max_out_len=len(prompts[0]) + 10,
    argmax_greedy= True,
    debug = True,
    request_activations= [lm.layer_name_format.format(i) for i in range(27,30)], #lm.model.config.n_layer
    use_cache=False,
    top_k = 20,
)
txt, ret_dict

/home/franlucc/codetrace/notebooks/../trace_model.py:135: UserWarning: The model `bigcode/starcoder` of type `gpt_bigcode` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.
  warnings.warn(f"The model `{self.model_name}` of type `{self.model_type}` already implements or can't utilize `use_cache` for fast generation. Setting `use_cache = False`.")


prompt <0> ==> [('2', 0.2827), ('1', 0.2601), ('3', 0.1105), (' world', 0.0573), ('4', 0.0417), ('0', 0.0287), ('5', 0.0235), ('6', 0.0158), (' "', 0.013), (' ")', 0.0104), ('7', 0.0086), ('9', 0.0075), ('世界', 0.0066), ('8', 0.0062), ('你', 0.0056), (' ",', 0.0052), (' from', 0.0045), ('我', 0.0038), (' "+', 0.0036), (' World', 0.0033)]
prompt <0> ==> [('")', 0.7178), ('.', 0.0358), ('!")', 0.0302), ('2', 0.0257), ('0', 0.0251), ('",', 0.0185), ('");', 0.0143), ('3', 0.0108), ('nd', 0.0092), (' ")', 0.008), ('"', 0.0073), ('1', 0.007), ('4', 0.0038), ('\\', 0.0036), (':', 0.0035), (',', 0.0033), ('5', 0.0033), (' from', 0.0031), (' world', 0.0029), (' "', 0.0025)]
prompt <0> ==> [('\n', 0.5107), ('<|endoftext|>', 0.137), ('\n\n', 0.0824), ('\n   ', 0.0535), ('\n       ', 0.0364), ('\r', 0.0193), ('\r\n\r', 0.0161), ('\r\n', 0.0136), ('\n\n   ', 0.0135), ('\n           ', 0.0094), ('\n\t', 0.0084), ('\n\n\n', 0.0072), ('\r\n   ', 0.0054), ('\n ', 0.0048), ('\n\n       ', 0.0043), (' #', 0

(['print("hello 2")\n<|endoftext|><fim_prefix><fim_suffix>e_name=\'ID\')),\n                (\'name\', models.CharField'],
 {'input_tokenized': [[('print', 1216),
    ('("', 440),
    ('hello', 7656),
    (' ', 225)]],
  'generated_tokens': [[[{'token': '2', 'id': 36, 'p': 0.28274163603782654},
     {'token': '1', 'id': 35, 'p': 0.2600540816783905},
     {'token': '3', 'id': 37, 'p': 0.11047524213790894},
     {'token': ' world', 'id': 5788, 'p': 0.05730068311095238},
     {'token': '4', 'id': 38, 'p': 0.04171457886695862},
     {'token': '0', 'id': 34, 'p': 0.028684914112091064},
     {'token': '5', 'id': 39, 'p': 0.023518485948443413},
     {'token': '6', 'id': 40, 'p': 0.015835966914892197},
     {'token': ' "', 'id': 313, 'p': 0.013031044974923134},
     {'token': ' ")', 'id': 7788, 'p': 0.010401885025203228},
     {'token': '7', 'id': 41, 'p': 0.008593253791332245},
     {'token': '9', 'id': 43, 'p': 0.007487054448574781},
     {'token': '世界', 'id': 33181, 'p': 0.00655871024355292

## Logit lens

In [5]:
lm.get_logits(prompts)


--- Argument Model Logit Lens ---
0: [(' ', 1), ('1', 0), ('<fim_suffix>', 0), (' (', 0), ('2', 0)]
1: [(' ', 1), ('1', 0), ('2', 0), ('<fim_suffix>', 0), (' "', 0)]
2: [(' ', 1), ('1', 0), ('2', 0), (' "', 0), (' (', 0)]
3: [(' ', 1), ('1', 1), ('2', 0), (' "', 0), (' (', 0)]
4: [(' ', 1), ('1', 1), ('2', 0), (' -', 0), (' (', 0)]
5: [('1', 1), (' ', 1), ('2', 0), (' -', 0), (' "', 0)]
6: [('1', 1), (' ', 1), (' -', 1), ('2', 0), (' "', 0)]
7: [('1', 1), (' -', 1), ('2', 1), (' ', 1), (' (', 1)]
8: [(' -', 2), ('1', 2), (' ', 1), ('2', 1), (' "', 1)]
9: [('1', 3), (' -', 2), (' world', 2), (' ', 2), ('2', 2)]
10: [('1', 3), (' -', 3), ('2', 2), (' ', 2), (' "', 1)]
11: [('1', 4), ('2', 3), (' world', 2), (' ', 2), ('3', 1)]
12: [('1', 7), ('2', 5), (' world', 3), ('3', 2), (' ', 2)]
13: [('1', 11), ('2', 8), (' world', 7), ('3', 3), ('world', 2)]
14: [('1', 11), (' world', 7), ('2', 7), ('3', 2), ('4', 1)]
15: [('1', 17), (' world', 15), ('2', 10), ('3', 3), ('world', 2)]
16: [(' wor

## Trace with patch

In [6]:
heads_to_save = [(20, 27)]
heads_to_patch = []
for l in range(0, 40):
    heads_to_patch += [(i,l) for i in range(48) if (i, l) not in heads_to_save]
print(heads_to_patch)

[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (15, 2), (16, 2), (17, 2), (18

In [55]:
prompts = ["x = 2 + "]
p = lm.trace_with_patch(prompts, heads_to_patch=[(h, lm.layername(l)) for (h,l) in heads_to_save], replace=False, noise = 0.9)

transformer.wte torch.Size([1, 6, 6144])
torch.Size([1, 6, 6144]) transformer.h.0
torch.Size([1, 6, 6144]) transformer.h.1
torch.Size([1, 6, 6144]) transformer.h.2
torch.Size([1, 6, 6144]) transformer.h.3
torch.Size([1, 6, 6144]) transformer.h.4
torch.Size([1, 6, 6144]) transformer.h.5
torch.Size([1, 6, 6144]) transformer.h.6
torch.Size([1, 6, 6144]) transformer.h.7
torch.Size([1, 6, 6144]) transformer.h.8
torch.Size([1, 6, 6144]) transformer.h.9
torch.Size([1, 6, 6144]) transformer.h.10
torch.Size([1, 6, 6144]) transformer.h.11
torch.Size([1, 6, 6144]) transformer.h.12
torch.Size([1, 6, 6144]) transformer.h.13
torch.Size([1, 6, 6144]) transformer.h.14
torch.Size([1, 6, 6144]) transformer.h.15
torch.Size([1, 6, 6144]) transformer.h.16
torch.Size([1, 6, 6144]) transformer.h.17
torch.Size([1, 6, 6144]) transformer.h.18
torch.Size([1, 6, 6144]) transformer.h.19
torch.Size([1, 6, 6144]) transformer.h.20
torch.Size([1, 6, 6144]) transformer.h.21
torch.Size([1, 6, 6144]) transformer.h.22
tor

In [56]:
lm.tokenizer.decode(p.argmax(dim=0))

')'

In [ ]:
prompt1 = ["x = 2 + "]
heads1 = lm.search_causal_heads(prompt1, layers=range(15,39), split=4)

## Search algorthim (stupid version)

- data: python integer completions (5 to start)
- search layer: middle 20-30
- search strategy: 8 heads x layer (40 tot searches)
- metric: top 1 output is not integer
- noise:replace

In [46]:
prompt1 = ["x = 2 + "]
heads1 = lm.search_causal_heads(prompt1, layers=range(15,39), split=4)

prompt2 = ["for i in range("]
heads2 = lm.search_causal_heads(prompt2, layers=range(15,39), split=4)

prompt3 = ["num = list(range(1,10))\nten = num["]
heads3 = lm.search_causal_heads(prompt3, layers=range(15,39), split=4)

3 [(0, 'transformer.h.15'), (1, 'transformer.h.15'), (2, 'transformer.h.15'), (3, 'transformer.h.15')]
3 [(4, 'transformer.h.15'), (5, 'transformer.h.15'), (6, 'transformer.h.15'), (7, 'transformer.h.15')]
3 [(8, 'transformer.h.15'), (9, 'transformer.h.15'), (10, 'transformer.h.15'), (11, 'transformer.h.15')]
3 [(12, 'transformer.h.15'), (13, 'transformer.h.15'), (14, 'transformer.h.15'), (15, 'transformer.h.15')]
3 [(16, 'transformer.h.15'), (17, 'transformer.h.15'), (18, 'transformer.h.15'), (19, 'transformer.h.15')]
2 [(20, 'transformer.h.15'), (21, 'transformer.h.15'), (22, 'transformer.h.15'), (23, 'transformer.h.15')]
1 [(24, 'transformer.h.15'), (25, 'transformer.h.15'), (26, 'transformer.h.15'), (27, 'transformer.h.15')]
2 [(28, 'transformer.h.15'), (29, 'transformer.h.15'), (30, 'transformer.h.15'), (31, 'transformer.h.15')]
1 [(32, 'transformer.h.15'), (33, 'transformer.h.15'), (34, 'transformer.h.15'), (35, 'transformer.h.15')]
9 [(36, 'transformer.h.15'), (37, 'transformer.

In [35]:
list(set(heads1).intersection(set(heads2)).intersection(set(heads3)))

[(30, 'transformer.h.29'),
 (45, 'transformer.h.2'),
 (39, 'transformer.h.12'),
 (28, 'transformer.h.30'),
 (26, 'transformer.h.6'),
 (17, 'transformer.h.16'),
 (28, 'transformer.h.16'),
 (40, 'transformer.h.31'),
 (18, 'transformer.h.2'),
 (43, 'transformer.h.3'),
 (40, 'transformer.h.1'),
 (38, 'transformer.h.5'),
 (41, 'transformer.h.23'),
 (32, 'transformer.h.28'),
 (37, 'transformer.h.0'),
 (25, 'transformer.h.22'),
 (11, 'transformer.h.18'),
 (38, 'transformer.h.24'),
 (38, 'transformer.h.29'),
 (30, 'transformer.h.27'),
 (32, 'transformer.h.34'),
 (14, 'transformer.h.26'),
 (41, 'transformer.h.32'),
 (34, 'transformer.h.6'),
 (37, 'transformer.h.33'),
 (0, 'transformer.h.13'),
 (0, 'transformer.h.5'),
 (11, 'transformer.h.5'),
 (3, 'transformer.h.23'),
 (45, 'transformer.h.6'),
 (25, 'transformer.h.9'),
 (20, 'transformer.h.17'),
 (9, 'transformer.h.8'),
 (31, 'transformer.h.17'),
 (0, 'transformer.h.14'),
 (39, 'transformer.h.5'),
 (41, 'transformer.h.25'),
 (3, 'transformer.h.

In [40]:
heads2

[(0, 'transformer.h.0'),
 (1, 'transformer.h.0'),
 (2, 'transformer.h.0'),
 (3, 'transformer.h.0'),
 (4, 'transformer.h.0'),
 (5, 'transformer.h.0'),
 (6, 'transformer.h.0'),
 (7, 'transformer.h.0'),
 (8, 'transformer.h.0'),
 (9, 'transformer.h.0'),
 (10, 'transformer.h.0'),
 (11, 'transformer.h.0'),
 (12, 'transformer.h.0'),
 (13, 'transformer.h.0'),
 (14, 'transformer.h.0'),
 (15, 'transformer.h.0'),
 (16, 'transformer.h.0'),
 (17, 'transformer.h.0'),
 (18, 'transformer.h.0'),
 (19, 'transformer.h.0'),
 (20, 'transformer.h.0'),
 (21, 'transformer.h.0'),
 (22, 'transformer.h.0'),
 (23, 'transformer.h.0'),
 (24, 'transformer.h.0'),
 (25, 'transformer.h.0'),
 (26, 'transformer.h.0'),
 (27, 'transformer.h.0'),
 (28, 'transformer.h.0'),
 (29, 'transformer.h.0'),
 (30, 'transformer.h.0'),
 (31, 'transformer.h.0'),
 (32, 'transformer.h.0'),
 (33, 'transformer.h.0'),
 (34, 'transformer.h.0'),
 (35, 'transformer.h.0'),
 (36, 'transformer.h.0'),
 (37, 'transformer.h.0'),
 (38, 'transformer.h.0

In [44]:
heads1

[(16, 'transformer.h.0'),
 (17, 'transformer.h.0'),
 (18, 'transformer.h.0'),
 (19, 'transformer.h.0'),
 (24, 'transformer.h.0'),
 (25, 'transformer.h.0'),
 (26, 'transformer.h.0'),
 (27, 'transformer.h.0'),
 (28, 'transformer.h.0'),
 (29, 'transformer.h.0'),
 (30, 'transformer.h.0'),
 (31, 'transformer.h.0'),
 (32, 'transformer.h.0'),
 (33, 'transformer.h.0'),
 (34, 'transformer.h.0'),
 (35, 'transformer.h.0'),
 (36, 'transformer.h.0'),
 (37, 'transformer.h.0'),
 (38, 'transformer.h.0'),
 (39, 'transformer.h.0'),
 (40, 'transformer.h.0'),
 (41, 'transformer.h.0'),
 (42, 'transformer.h.0'),
 (43, 'transformer.h.0'),
 (44, 'transformer.h.0'),
 (45, 'transformer.h.0'),
 (46, 'transformer.h.0'),
 (47, 'transformer.h.0'),
 (0, 'transformer.h.1'),
 (1, 'transformer.h.1'),
 (2, 'transformer.h.1'),
 (3, 'transformer.h.1'),
 (4, 'transformer.h.1'),
 (5, 'transformer.h.1'),
 (6, 'transformer.h.1'),
 (7, 'transformer.h.1'),
 (8, 'transformer.h.1'),
 (9, 'transformer.h.1'),
 (10, 'transformer.h.1

In [42]:
lm.get_logits(["x = 2 + "])


--- Argument Model Logit Lens ---
0: [(' ', 2), ('<fim_suffix>', 1), ('\n', 0), (' (', 0), ('1', 0)]
1: [(' ', 1), ('<fim_suffix>', 0), ('1', 0), (' (', 0), ('\n', 0)]
2: [(' ', 1), ('1', 1), ('<fim_suffix>', 1), (' (', 0), ('\n', 0)]
3: [(' ', 1), ('1', 1), (' (', 1), ('<fim_suffix>', 1), ('\n', 0)]
4: [(' ', 1), (' (', 1), ('1', 1), ('\n', 1), ('<fim_suffix>', 0)]
5: [(' ', 2), ('1', 2), (' (', 1), ('\n', 1), ('2', 1)]
6: [('1', 2), (' (', 1), (' ', 1), ('2', 1), ('3', 0)]
7: [('1', 3), (' ', 2), (' (', 2), ('2', 1), ('3', 0)]
8: [(' (', 2), ('1', 2), (' ', 1), ('2', 1), ('3', 1)]
9: [('1', 2), (' (', 2), (' ', 2), ('2', 1), ('3', 1)]
10: [(' (', 2), ('1', 2), (' ', 2), ('2', 1), ('<fim_middle>', 1)]
11: [('1', 3), (' (', 2), (' ', 2), ('2', 2), ('3', 1)]
12: [('1', 5), ('2', 3), (' (', 3), (' ', 2), ('3', 1)]
13: [('1', 7), ('2', 4), ('3', 2), (' (', 1), ('4', 1)]
14: [('1', 9), ('2', 4), (' ', 3), ('3', 2), (' (', 2)]
15: [('1', 10), ('2', 7), (' ', 3), ('4', 3), (' (', 2)]
16: [(

In [50]:
heads3

[(32, 'transformer.h.15'),
 (33, 'transformer.h.15'),
 (34, 'transformer.h.15'),
 (35, 'transformer.h.15'),
 (36, 'transformer.h.15'),
 (37, 'transformer.h.15'),
 (38, 'transformer.h.15'),
 (39, 'transformer.h.15'),
 (40, 'transformer.h.15'),
 (41, 'transformer.h.15'),
 (42, 'transformer.h.15'),
 (43, 'transformer.h.15'),
 (28, 'transformer.h.16'),
 (29, 'transformer.h.16'),
 (30, 'transformer.h.16'),
 (31, 'transformer.h.16'),
 (32, 'transformer.h.16'),
 (33, 'transformer.h.16'),
 (34, 'transformer.h.16'),
 (35, 'transformer.h.16'),
 (36, 'transformer.h.16'),
 (37, 'transformer.h.16'),
 (38, 'transformer.h.16'),
 (39, 'transformer.h.16'),
 (40, 'transformer.h.16'),
 (41, 'transformer.h.16'),
 (42, 'transformer.h.16'),
 (43, 'transformer.h.16'),
 (24, 'transformer.h.17'),
 (25, 'transformer.h.17'),
 (26, 'transformer.h.17'),
 (27, 'transformer.h.17'),
 (32, 'transformer.h.17'),
 (33, 'transformer.h.17'),
 (34, 'transformer.h.17'),
 (35, 'transformer.h.17'),
 (40, 'transformer.h.17'),
 

In [47]:
set(heads1).intersection(set(heads2)).intersection(set(heads3))

{(0, 'transformer.h.19'),
 (0, 'transformer.h.22'),
 (0, 'transformer.h.23'),
 (0, 'transformer.h.27'),
 (1, 'transformer.h.19'),
 (1, 'transformer.h.22'),
 (1, 'transformer.h.23'),
 (1, 'transformer.h.27'),
 (2, 'transformer.h.19'),
 (2, 'transformer.h.22'),
 (2, 'transformer.h.23'),
 (2, 'transformer.h.27'),
 (3, 'transformer.h.19'),
 (3, 'transformer.h.22'),
 (3, 'transformer.h.23'),
 (3, 'transformer.h.27'),
 (4, 'transformer.h.18'),
 (4, 'transformer.h.27'),
 (5, 'transformer.h.18'),
 (5, 'transformer.h.27'),
 (6, 'transformer.h.18'),
 (6, 'transformer.h.27'),
 (7, 'transformer.h.18'),
 (7, 'transformer.h.27'),
 (8, 'transformer.h.18'),
 (8, 'transformer.h.19'),
 (8, 'transformer.h.20'),
 (8, 'transformer.h.21'),
 (8, 'transformer.h.22'),
 (8, 'transformer.h.23'),
 (8, 'transformer.h.26'),
 (8, 'transformer.h.27'),
 (9, 'transformer.h.18'),
 (9, 'transformer.h.19'),
 (9, 'transformer.h.20'),
 (9, 'transformer.h.21'),
 (9, 'transformer.h.22'),
 (9, 'transformer.h.23'),
 (9, 'transf

In [51]:
lm.get_logits(prompt2)


--- Argument Model Logit Lens ---
0: [('(', 1), (' ', 1), (' (', 1), ('<fim_suffix>', 0), ('\n', 0)]
1: [(' ', 1), ('(', 1), (' (', 0), ('\n', 0), ('<fim_suffix>', 0)]
2: [(' ', 2), (' (', 1), ('(', 1), ('s', 1), ('\n', 1)]
3: [(' ', 3), (' (', 2), ('s', 1), ('\n', 1), ('(', 1)]
4: [(' ', 5), (' (', 2), ('s', 1), ('\n', 1), ('(', 1)]
5: [(' ', 6), (' (', 3), ('s', 1), ('1', 1), ('0', 1)]
6: [(' ', 8), (' (', 3), ('s', 2), ('0', 2), ('1', 1)]
7: [(' ', 12), (' (', 4), ('0', 3), ('1', 2), ('s', 2)]
8: [(' ', 9), ('0', 4), (' (', 3), ('1', 3), ('s', 2)]
9: [(' ', 7), ('0', 4), ('1', 3), (' (', 3), ('s', 2)]
10: [(' ', 8), ('0', 6), ('1', 5), (' (', 3), ('2', 2)]
11: [(' ', 11), ('1', 9), ('0', 7), (' (', 4), ('3', 3)]
12: [(' ', 18), ('1', 12), ('0', 11), (' (', 4), ('3', 3)]
13: [(' ', 22), ('1', 12), ('0', 10), (' (', 4), ('len', 3)]
14: [(' ', 21), ('1', 13), ('0', 12), ('len', 6), (' (', 3)]
15: [(' ', 21), ('1', 19), ('0', 12), ('len', 9), (' (', 3)]
16: [(' ', 21), ('len', 20), ('1